In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import LabelEncoder

#classification algorithms
from sklearn.ensemble import RandomForestClassifier

#Evaluation metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [3]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Hl4ocndD-VcKyZvX8MoaqtyxCaBYn5Pk2dQ-qHzulCx9',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'projectckd-donotdelete-pr-mokhltuq7dllkt'
object_key = 'data.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)

In [4]:
df.head()

,blood_pressure,specific_gravity,albumin,sugar,blood_urea,serum_creatinine,sodium,potassium,hemoglobin,packed_cell_volume,white_blood_cell_count,hypertension,diabetesmellitus,coronary_artery_disease,age,class
0,80.0,1.020,1.0,0.0,36.0,1.2,138.0,4.4,15.4,44,7800.0,1,1,0,48,0
1,50.0,1.020,4.0,0.0,18.0,0.8,138.0,4.4,11.3,38,6000.0,0,0,0,7,0
2,80.0,1.010,2.0,3.0,53.0,1.8,138.0,4.4,9.6,31,7500.0,0,1,0,62,0
3,70.0,1.005,4.0,0.0,56.0,3.8,111.0,2.5,11.2,32,6700.0,1,0,0,48,0
4,80.0,1.010,2.0,0.0,26.0,1.4,138.0,4.4,11.6,35,7300.0,0,0,0,51,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   blood_pressure           400 non-null    float64
 1   specific_gravity         400 non-null    float64
 2   albumin                  400 non-null    float64
 3   sugar                    400 non-null    float64
 4   blood_urea               400 non-null    float64
 5   serum_creatinine         400 non-null    float64
 6   sodium                   400 non-null    float64
 7   potassium                400 non-null    float64
 8   hemoglobin               400 non-null    float64
 9   packed_cell_volume       400 non-null    int64  
 10  white_blood_cell_count   400 non-null    float64
 11  hypertension             400 non-null    int64  
 12  diabetesmellitus         400 non-null    int64  
 13  coronary_artery_disease  400 non-null    int64  
 14  age                      4

In [6]:
df.isnull().sum()

blood_pressure             0
specific_gravity           0
albumin                    0
sugar                      0
blood_urea                 0
serum_creatinine           0
sodium                     0
potassium                  0
hemoglobin                 0
packed_cell_volume         0
white_blood_cell_count     0
hypertension               0
diabetesmellitus           0
coronary_artery_disease    0
age                        0
class                      0
dtype: int64

In [7]:
X = df.drop(["class"], axis=1)
y = df["class"]

In [8]:
X

,blood_pressure,specific_gravity,albumin,sugar,blood_urea,serum_creatinine,sodium,potassium,hemoglobin,packed_cell_volume,white_blood_cell_count,hypertension,diabetesmellitus,coronary_artery_disease,age
0,80.0,1.020,1.0,0.0,36.0,1.2,138.0,4.4,15.4,44,7800.0,1,1,0,48
1,50.0,1.020,4.0,0.0,18.0,0.8,138.0,4.4,11.3,38,6000.0,0,0,0,7
2,80.0,1.010,2.0,3.0,53.0,1.8,138.0,4.4,9.6,31,7500.0,0,1,0,62
3,70.0,1.005,4.0,0.0,56.0,3.8,111.0,2.5,11.2,32,6700.0,1,0,0,48
4,80.0,1.010,2.0,0.0,26.0,1.4,138.0,4.4,11.6,35,7300.0,0,0,0,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,80.0,1.020,0.0,0.0,49.0,0.5,150.0,4.9,15.7,47,6700.0,0,0,0,55
396,70.0,1.025,0.0,0.0,31.0,1.2,141.0,3.5,16.5,54,7800.0,0,0,0,42
397,80.0,1.020,0.0,0.0,26.0,0.6,137.0,4.4,15.8,49,6600.0,0,0,0,12
398,60.0,1.025,0.0,0.0,50.0,1.0,135.0,4.9,14.2,51,7200.0,0,0,0,17


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [10]:
X_train

,blood_pressure,specific_gravity,albumin,sugar,blood_urea,serum_creatinine,sodium,potassium,hemoglobin,packed_cell_volume,white_blood_cell_count,hypertension,diabetesmellitus,coronary_artery_disease,age
3,70.0,1.005,4.0,0.0,56.0,3.8,111.0,2.5,11.2,32,6700.0,1,0,0,48
18,100.0,1.025,0.0,3.0,27.0,1.3,135.0,4.3,12.7,37,11400.0,1,1,1,60
202,60.0,1.020,0.0,0.0,74.0,2.9,135.0,5.9,8.0,24,8000.0,0,1,0,78
250,80.0,1.025,0.0,0.0,10.0,1.2,135.0,5.0,15.0,48,10400.0,0,0,0,40
274,80.0,1.020,0.0,0.0,23.0,0.7,141.0,4.2,14.4,44,8000.0,0,0,0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,60.0,1.010,1.0,0.0,92.0,3.3,141.0,4.0,9.8,28,14600.0,1,1,0,46
106,90.0,1.020,0.0,0.0,118.0,6.1,127.0,4.4,6.0,17,6500.0,1,1,0,50
270,80.0,1.025,0.0,0.0,34.0,1.1,145.0,4.0,14.3,41,7200.0,0,0,0,23
348,80.0,1.020,0.0,0.0,19.0,0.5,147.0,3.5,13.6,44,7300.0,0,0,0,38


In [11]:
ckd_RF = RandomForestClassifier()
ckd_RF.fit(X_train.values, Y_train.values)

RandomForestClassifier()

In [12]:
predictions = ckd_RF.predict(X_test.values)

In [13]:
pred

NameError: name 'pred' is not defined

In [21]:
ckd_RF.predict([[80.0,	1.020,	0.0,	0.0,	49.0,	0.5,	150.0,	4.9,	15.7,	47,	6700.0	0	0	0	55]])

array([0])

In [ ]:
ckd_RF_acc = accuracy_score(Y_test.values, pred)
print(classification_report(Y_test.values, pred))
print(confusion_matrix(Y_test.values, pred))

In [ ]:
!pip install ibm_watson_machine_learning

In [14]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey":"_YBwCIVTCKowMStvz4HHQTcizrYGJUV0p9bj_W4uzh3-"
                  }

client = APIClient(wml_credentials)

In [15]:
def guid_from_space_name(client, space_name):
 space = client.spaces.get_details()
 return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [16]:
 space_uid = guid_from_space_name(client, "models")
 print("Space UID = " , space_uid)

Space UID =  6da0b622-c7a0-415f-bce4-b79015e1320b


In [17]:
 client.set.default_space(space_uid)

'SUCCESS'

In [18]:
 client.software_specifications.list(limit=100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [19]:
software_spec_id =  client.software_specifications.get_id_by_name('runtime-22.1-py3.9')
print(software_spec_id)

12b83a17-24d8-5082-900f-0ab31fbfd3cb


In [20]:
model_details = client.repository.store_model(model=ckd_RF,meta_props={
client.repository.ModelMetaNames.NAME:"PROJECT_CKD",
client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_id})
model_id = client.repository.get_model_id(model_details)